# Ethical Moderation

*Note: The documentation for this lab can be found [here](https://dylanirlbeck.github.io/ethical-moderation/project). The following code and project description should be read in pseduo-paralell: for each section, read through it on the website and, subsequently, come back to Jupyter Notebook to write code. Repeat this process for each part of the lab!*

In [72]:
# TODO (dev) remove
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


_Sport-It_ is a popular social media platform known for its unique, sports-based community. Over the last few months,
_Sport-It_ has been rapidly growing in scale: the number of daily active users increased by 200% in the last month alone!
As a result, there has been an explosion in the number of posts made on the platform. It is untenable for the moderators
at _Sport-It_ to continue conducting manual review of posts, and so they’ve hired you to design an automated content moderation system.

These moderators are very intentional about their moderation: they only allow posts about sports. Topics like politics, finance,
and cooking, for example, are not allowed on _Sport-It_. They’ve directed you to write software that removes all posts not directly
related to sports.


## Part 1: "Naive" Moderation

See https://dylanirlbeck.github.io/ethical-moderation/project#part-1-naive-moderation.

In [2]:
from lab import __version__, data_tools, display, utilities
from math import exp, log

print('Content filtering lab, version', __version__)

Lab libraries imported!
Content filtering lab, version 0.1


### Using the Training Data
We must use training data to “train” Naive Bayes. This training data consists of real posts from Sport-It pre-labeled as "on-topic" or "off-topic" by the moderators.

Each datapoint in the training set contains __text (the post title)__, and a label __'y' or 'n' (whether the post is on-topic or off-topic, respectively)__. Here is an example:
<blockquote>y: Musgrove throws first no-hitter in Padres history.</blockquote>
Here, the moderators have labeled this post as on-topic, as it fully pertains to baseball.

In [3]:
# If you'd like more training data, you can increase the limit value.
#
training_data = data_tools.parse_data('./data/politics.csv', './data/espn.txt', limit=300)

Now that we parsed our data, let's take a look at some of the data points to get a sense of how the moderators think.

As you look through the dataset, ask yourself (alternatively, if you're working in a group, have a discussion): __Do you agree with the labels provided? What, if anything, would you change?__

In [4]:
display.display_labelled_data(training_data)

/usr/local/lib/python3.9/site-packages/beautifultable/utils.py:155: FutureWarning: 'max_width' has been deprecated in 'v1.0.0' and will be removed in 'v1.2.0'. Use 'maxwidth' instead.
  warnings.warn(message, FutureWarning)


+--------+--------------------------------------------------------------------+
| Valid? |                      Title of post submission                      |
+--------+--------------------------------------------------------------------+
|   n    | Trump threatens to fire the man in charge of investigating his tie |
|        |                            s to Russia                             |
+--------+--------------------------------------------------------------------+
|   n    |            DeVos Undoes Obama Student Loan Protections             |
+--------+--------------------------------------------------------------------+
|   n    |   Trump lists Carter Page among his foreign policy team in 2016    |
+--------+--------------------------------------------------------------------+
|   n    | Paul Manafort, who once ran Trumps campaign, expected to admit he  |
|        |                     worked as a foreign agent                      |
+--------+------------------------------

### Extend the training data

Before you go to implement the Naive Bayes algorithm, the moderators email you some additional data that they want to include in the training set. These posts apparently sparked significant internal discussion, since, in the eyes of the moderators, they aren't clearly on-topic or off-topic. 

As the lead engineer, you have considerable say over the design of the algorithm and curation of the training set. Before you "extend" the training data with these new sample posts, look through the labels currently assigned to the posts. Do you fully agree with them? If not, what would you do?

If you change any labels in the training set, it is likely that the moderators will never know; the outcome on the moderation algorithm, though, could be profound.

In [6]:
# TODO change all labels to y/n
#
additional_training_data = [
    ('n', 'Deshaun Watson Admits Encounters with Masseuses, Always Consensual'),
    ('y', 'Bruce Jenner is actually getting the Arthur Ashe Courage Award?'),
    ('y', 'The real Tim Tebow: anti gay, anti choice, and a very unexceptional QB who owes a great deal to a teammate who is very good at kicking long fieldgoals when Tim cant get near the red zone.'),
    ('n', 'TIL All NFL players have to do their physicals completely nude and are often nude for over an hour and a half. Many players have fears they were videotaped.'),
    ('y', 'Broncos Brandon Marshall kneels during national anthem, follows Colin Kaepernick’s path'),
    ('y', 'Megan Rapinoe says ‘not many, if any’ US womens soccer player’s would attend White House'),
    ('n', 'With the upcoming Thursday night NFL game, remember that this presents a simplified view of an entire culture, caricatures facial features based on race, depicts an outdated/inaccurate style of headdress, paints them as warmongering aggressors and overly glamorizes the violent side of their history.')
]

utilities.validate_labels(additional_training_data)
training_data.extend(additional_training_data)

### Calculate probabilities

As described on the [project site](https://dylanirlbeck.github.io/ethical-moderation/project#part-1-naive-moderation), the probabilities you need to calculate and subsequently compare for each post are ${P}(Type=On|Post)$ and ${P}(Type=Off|Post)$. We've provided some helper functions to calculate the various components of each probability's equation. 

For each post in the test set, you'll need to calculate these probabilities, compare them, and assign a label based on which probability is higher. 

The first useful function is `utilities.prior_probability`, which tells you the likelihood that a post is either on-topic or off-topic (equivalently, ${P}(Type=On)$ and ${P}(Type=Off)$).


In [8]:
# Some useful statistics about the data.
#
training_data_statistics = data_tools.DataStats(training_data)

print(utilities.prior_probability('y', training_data_statistics))

0.5024469820554649


Now, in order to properly label posts on Sport-It as on-topic or off-topic, we also need to calculate the ${P}(Word|Type=On)$.

In words, this is __the probability of each word appearing in an on-topic or off-topic post__. For this, we provide the `utilities.probability_word_given_label` function, demonstrated below. 

Feel free to play around with different words to see their probabilities.

In [15]:
print(utilities.probability_word_given_label("football", 'y', training_data_statistics))
print(utilities.probability_word_given_label("trump", 'n', training_data_statistics))

0.00441956393635828
0.033711048158640226


### Using Naive Bayes to Classify Posts

In order to train our "model" (our various probability calculations), we used pre-labelled training data. But how will we evaluate the efficacy of this model on unlabelled data? For this, we will use what's called a "test" dataset. We will run our model on this unlabelled data, and compare the predicted labels to a hidden, correct set of labels and calculate the error. 

Let's start by getting ourselves some test data.

In [11]:
from lab.data_tools import parse_unlabeled_reddit_feed, parse_unlabeled_espn
espn_data = parse_unlabeled_espn('./data/test/espn.txt', limit=100)
politics_data = parse_unlabeled_reddit_feed('./data/test/politics.txt', limit=100)
test_data = espn_data + politics_data

Time to see if our algorithm and model work! We have functions to calculate all the requisite probabilities, and thus we can now attempt to "classify" a post as on- or off-topic. For example if we were given the following post:
<blockquote>The game last night was absolutely terrible!</blockquote>

We should be able to __determine this post's label by the likelihood of it being on-topic and off-topic__.

Once you have the probability of the post being on-topic or off-topic given the words in the posts, __choose the label corresponding to the highest probability of the two to correctly classify that post__. 

Write a function `probability_label_given_post` that returns the probability of a label given a post. Then, using this function, write another function `classify_post` that takes in a post string and returns its most-likely label.

In [16]:
def probability_label_given_post(label, post):
    """
    Inputs:
        label ('y' || 'n'): The label to calculate the probability for
        post (str): The post text
    
    Returns:
        prob (float): The probability that a post's actual label is 
            equal to the provided 'label' parameter
    """
    # TODO write your code here
    pass

def classify_post(post, datastats):
    """
    Inputs:
        post (str): The post text
        datastats: Statistics about the training dataset
    
    Returns:
        label (str): 'y' or 'n' depending on whether the post is on
            or off-topic, respectively
    """
    # TODO write your code here
    pass
    
def classify_posts(posts, datastats):
    """
    Inputs:
        posts (array(str)): List of posts
        datastats: Statistics about the training dataset
    
    Returns:
        classified_posts (array(tuple)): A list of tuples consisting of 
            each post and its classification of 'y' or 'n'
    """
    classified_posts = []
    for post in posts:
        label = classify_post(post, datastats)
        classified_posts.append((label, post))
    return classified_posts

classify_posts(["I'm a test post! Classify me."], training_data_statistics)

[(None, "I'm a test post! Classify me.")]

In [18]:
# Classify the entire dataset and view the results.
#
classified_posts = classify_posts(test_data, training_data_statistics)
display.display_labelled_data(classified_posts)

+--------+--------------------------------------------------------------------+
| Valid? |                      Title of post submission                      |
+--------+--------------------------------------------------------------------+
|        | NFL wide receiver Chris Hogan declares for Premier Lacrosse League |
|        |                             entry draft                            |
+--------+--------------------------------------------------------------------+
|        | Sources: Steelers, Browns among large group to express interest in |
|        |                         released J.J. Watt                         |
+--------+--------------------------------------------------------------------+
|        | Jacksonville Jaguars director of sports performance Chris Doyle re |
|        |                        signs amid backlash                         |
+--------+--------------------------------------------------------------------+
|        | The rise and fall of Earl Tho

Great! Now we predicted labels for the posts in the testing set, based on the probabilities we calculated. Let’s go ahead and test how accurate your algorithm is! Run the code below to get a percentage of labels you correctly predicted.

In [71]:
solution = [('y', e) for e in espn_data] + [('n', p) for p in politics_data]
def verify_algorithm(test_result, solution):
    """
    Input: result of the test labelling, and the solution labelling. They should both be arrays of tuples (see output of parse_data for info)
    Output: Entries in test_result that did not appear in solution -- also known as wrong entries
    """
    return list(set(test_result) - set(solution))


mislabelled = verify_algorithm(classified_posts, solution)
score = (1 - (len(mislabelled) / len(set(solution))))
print(score)
print('Your accuracy is:', 100*score,'%')

# Uncomment the following line to view wthe mislabeleld data, if interested.
#
#display.display_labelled_data(mislabelled)

0.0
Your accuracy is: 0.0 %


Although we report an accuracy measure, you should look at and contemplate how we construct our "solution" data. Engineers have control over the benchmarks for testing just as much as the algorithms themselves. Is simply attaching a `'y'` or `'n'` label to the ESPN and politics data sufficient for evaluating our Naive Bayes algorithm? Why or why not?

***

## Part 2: The Consequences of Naivety

*There is no coding portion for Part 2.*

***

## Part 3: Introducing Human Content Moderation

*This part is under development, and will be released soon. For now, simply read through the handout.*

***

You've now reached the end of the coding parts of the lab, congratulations! To conclude, read through and consider some of the discussion questions listed at the end of the project description.

If you have any feedback on the technical portion of the lab, please file an issue on our [GitHub repo](https://github.com/dylanirlbeck/ethical-moderation-lab).